# VLM Chart Pattern Analyzer - Google Colab Benchmark

This notebook runs VLM inference on chart images and outputs results to CSV.

Upload your chart images or mount Google Drive with images.

## Configuration

Set the parameters for the benchmark.

In [ ]:
#@title Benchmark Parameters
#@param {type:"string"}
model_id = "qwen2-vl-2b" #@param ["qwen2-vl-2b", "HuggingFaceTB/SmolVLM2-2.2B-Instruct", "other"] {allow-input: true}
#@param {type:"string"}
precision = "fp16" #@param ["fp32", "fp16", "int8", "int4"]
#@param {type:"boolean"}
use_gpu = True #@param {type:"boolean"}
#@param {type:"string"}
hf_token = "" #@param {type:"string"}
#@param {type:"integer"}
image_limit = 0 #@param {type:"integer"}

print(f"Model ID: {model_id}")
print(f"Precision: {precision}")
print(f"Use GPU: {use_gpu}")
print(f"Image limit: {image_limit}")

## Install Dependencies

In [ ]:
!pip install transformers torch torchvision torchaudio --quiet
!pip install huggingface_hub --quiet
!pip install pillow --quiet
!pip install accelerate --quiet
!pip install num2words --quiet

# Test imports
try:
    import num2words
    print("✓ num2words imported successfully")
except ImportError as e:
    print(f"✗ num2words import failed: {e}")

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Upload Images

Upload your chart images or mount Google Drive.

In [ ]:
from google.colab import files
import os
from pathlib import Path

# Create images directory
images_dir = Path("/content/chart_images")
images_dir.mkdir(exist_ok=True)

print("Upload your chart images (PNG, JPG, etc.)")
uploaded = files.upload()

image_paths = []
for filename, content in uploaded.items():
    filepath = images_dir / filename
    with open(filepath, 'wb') as f:
        f.write(content)
    image_paths.append(str(filepath))
    print(f"Uploaded: {filename}")

print(f"\nTotal images uploaded: {len(image_paths)}")
print(f"Images directory: {images_dir}")

# Alternative: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# images_dir = "/content/drive/MyDrive/path/to/your/chart/images"
# image_paths = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

## Load Model from HuggingFace

In [ ]:
import os
from huggingface_hub import login
from transformers import AutoProcessor, AutoModelForImageTextToText

print(f"Loading model: {model_id}")
print(f"Precision: {precision}")

# Authenticate if token provided
if hf_token:
    login(token=hf_token)
    print("Authenticated with HuggingFace")
else:
    print("No HF token provided - using public models only")

# Load processor
print("Loading processor...")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
print("✓ Processor loaded")

# Load model with appropriate dtype
torch_dtype = torch.float16 if precision == "fp16" else torch.float32
device = "cuda" if use_gpu and torch.cuda.is_available() else "cpu"

print(f"Loading model on {device} with dtype {torch_dtype}...")
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch_dtype
).to(device)

print("✓ Model loaded successfully")
print(f"Device: {device}")
print(f"Model dtype: {model.dtype}")

## Run Inference

In [ ]:
import csv
from datetime import datetime
from PIL import Image
import time

# Limit images if specified
if image_limit > 0:
    image_paths = image_paths[:image_limit]

print(f"Running inference on {len(image_paths)} images...")

results = []
model.eval()

for img_path in image_paths:
    print(f"Processing: {os.path.basename(img_path)}")

    row = {
        'image_filename': os.path.basename(img_path),
        'model_id': model_id,
        'precision': precision,
        'device': device,
        'timestamp': datetime.now().isoformat(),
        'response': None,
        'error': None,
        'latency_ms': None,
        'memory_used_mb': None,
        'tokens_generated': None,
        'throughput_tokens_per_sec': None
    }

    try:
        # Load and prepare image
        img = Image.open(img_path).convert('RGB')

        # Prepare inputs
        inputs = processor(images=img, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Reset memory stats
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

        # Run inference
        start_time = time.perf_counter()

        with torch.no_grad():
            generated = model.generate(**inputs, max_new_tokens=128)

        end_time = time.perf_counter()

        # Decode response
        response = processor.batch_decode(generated, skip_special_tokens=True)[0]

        # Calculate metrics
        latency_ms = (end_time - start_time) * 1000
        tokens_generated = generated.shape[-1] if hasattr(generated, 'shape') else len(generated[0]) if isinstance(generated, list) else None
        throughput = tokens_generated / (end_time - start_time) if tokens_generated else None

        memory_used_mb = None
        if torch.cuda.is_available():
            memory_used_mb = torch.cuda.max_memory_allocated() / (1024 * 1024)

        row.update({
            'response': response,
            'latency_ms': round(latency_ms, 3),
            'memory_used_mb': round(memory_used_mb, 3) if memory_used_mb else None,
            'tokens_generated': tokens_generated,
            'throughput_tokens_per_sec': round(throughput, 3) if throughput else None
        })

        print(f"  ✓ Completed in {latency_ms:.1f}ms")

    except Exception as e:
        row['error'] = str(e)
        print(f"  ✗ Error: {e}")

    results.append(row)

print(f"\nInference completed. Processed {len(results)} images.")

## Export Results to CSV

In [ ]:
csv_path = "/content/benchmark_results.csv"

fieldnames = [
    'image_filename', 'model_id', 'precision', 'device',
    'latency_ms', 'memory_used_mb', 'tokens_generated', 'throughput_tokens_per_sec',
    'timestamp', 'response', 'error'
]

with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"✓ Results saved to {csv_path}")
print(f"Total results: {len(results)}")

# Display download link
from google.colab import files
files.download(csv_path)

print("\nDownload the CSV file above and copy it to your repository.")

In [ ]:
from google.colab import files
files.download('/content/benchmark_results.csv')